# Part 1

As your flight slowly drifts through the sky, the Elves at the Mythical Information Bureau at the North Pole contact you. They'd like some help debugging a malfunctioning experimental energy source aboard one of their super-secret imaging satellites.

The experimental energy source is based on cutting-edge technology: a set of Conway Cubes contained in a pocket dimension! When you hear it's having problems, you can't help but agree to take a look.

The pocket dimension contains an infinite 3-dimensional grid. At every integer 3-dimensional coordinate `(x,y,z)`, there exists a single cube which is either active or inactive.

In the initial state of the pocket dimension, almost all cubes start inactive. The only exception to this is a small flat region of cubes (your puzzle input); the cubes in this region start in the specified active (`#`) or inactive (`.`) state.

The energy source then proceeds to boot up by executing six cycles.

Each cube only ever considers its neighbors: any of the 26 other cubes where any of their coordinates differ by at most 1. For example, given the cube at `x=1,y=2,z=3`, its neighbors include the cube at `x=2,y=2,z=2`, the cube at `x=0,y=2,z=3`, and so on.

During a cycle, all cubes simultaneously change their state according to the following rules:

- If a cube is active and exactly 2 or 3 of its neighbors are also active, the cube remains active. Otherwise, the cube becomes inactive.
- If a cube is inactive but exactly 3 of its neighbors are active, the cube becomes active. Otherwise, the cube remains inactive.

The engineers responsible for this experimental energy source would like you to simulate the pocket dimension and determine what the configuration of cubes should be at the end of the six-cycle boot process.

For example, consider the following initial state:
```
.#.
..#
###
```
Even though the pocket dimension is 3-dimensional, this initial state represents a small 2-dimensional slice of it. (In particular, this initial state defines a 3x3x1 region of the 3-dimensional space.)

Simulating a few cycles from this initial state produces the following configurations, where the result of each cycle is shown layer-by-layer at each given z coordinate (and the frame of view follows the active cells in each cycle):

Before any cycles:
```
z=0
.#.
..#
###
```

After 1 cycle:
```
z=-1
#..
..#
.#.

z=0
#.#
.##
.#.

z=1
#..
..#
.#.
```

After 2 cycles:
```
z=-2
.....
.....
..#..
.....
.....

z=-1
..#..
.#..#
....#
.#...
.....

z=0
##...
##...
#....
....#
.###.

z=1
..#..
.#..#
....#
.#...
.....

z=2
.....
.....
..#..
.....
.....
```

After 3 cycles:
```
z=-2
.......
.......
..##...
..###..
.......
.......
.......

z=-1
..#....
...#...
#......
.....##
.#...#.
..#.#..
...#...

z=0
...#...
.......
#......
.......
.....##
.##.#..
...#...

z=1
..#....
...#...
#......
.....##
.#...#.
..#.#..
...#...

z=2
.......
.......
..##...
..###..
.......
.......
.......
```
After the full six-cycle boot process completes, 112 cubes are left in the active state.

Starting with your given initial configuration, simulate six cycles. How many cubes are left in the active state after the sixth cycle?

In [1]:
from collections import namedtuple, defaultdict


Cube = namedtuple('Cube', ['x', 'y', 'z'])


def get_input():
    with open('inputs/16') as f:
        return f.read().splitlines()


class EnergySource:
    def __init__(self, init_state):
        self.active_cubes = self._get_initial_active_cubes(init_state)
    
    def _get_initial_active_cubes(self, init_state):
        active_cubes = set()
        for row_num, row in enumerate(init_state):
            for col_num, cube in enumerate(row):
                if cube == '#':
                    active_cubes.add(Cube(row_num, col_num, 0))
        return active_cubes
    
    def next_cycle(self):
        num_active_neighbors = defaultdict(int)
        for active in self.active_cubes:
            for neighb in self._get_neighbors(active):
                num_active_neighbors[neighb] += 1
        new_active_cubes = set()
        for cube, num_act_neighbs in num_active_neighbors.items():
            if num_act_neighbs == 3:
                new_active_cubes.add(cube)
            elif num_act_neighbs == 2 and cube in self.active_cubes:
                new_active_cubes.add(cube)
        self.active_cubes = new_active_cubes
    
    def _get_neighbors(self, cube):
        neighbors = []
        diffs = (-1, 0, 1)
        for x_diff in diffs:
            for y_diff in diffs:
                for z_diff in diffs:
                    if x_diff == 0 and y_diff == 0 and z_diff == 0:
                        continue
                    neighb = Cube(cube.x + x_diff, cube.y + y_diff, cube.z + z_diff)
                    neighbors.append(neighb)
        return neighbors
    
    def print_grid(self):
        'For testing and debugging.'
        x_coords = {c.x for c in self.active_cubes}
        y_coords = {c.y for c in self.active_cubes}
        z_coords = {c.z for c in self.active_cubes}
        for z in range(min(z_coords), max(z_coords)+1):
            print('z=%s' % z)
            for x in range(min(x_coords), max(x_coords)+1):
                row = ''
                for y in range(min(y_coords), max(y_coords)+1):
                    cube = '#' if Cube(x, y, z) in self.active_cubes else '.'
                    row += cube
                print(row)
            print('\n')
    

    
    
def test():
    'To check that it produces the same thing as in the example.'
    test_input = [
        '.#.',
        '..#',
        '###'
    ]
    e = EnergySource(test_input)
    e.print_grid()
    print('After 1 cycle\n')
    e.next_cycle()
    e.print_grid()
    print('After 2 cycles\n')
    e.next_cycle()
    e.print_grid()


# test()


def result1():
    init_state = get_input()
    e = EnergySource(init_state)
    for i in range(6):
        e.next_cycle()
    return len(e.active_cubes)


result1()

218

# Part 2

For some reason, your simulated results don't match what the experimental energy source engineers expected. Apparently, the pocket dimension actually has four spatial dimensions, not three.

The pocket dimension contains an infinite 4-dimensional grid. At every integer 4-dimensional coordinate `(x,y,z,w)`, there exists a single cube (really, a hypercube) which is still either active or inactive.

Each cube only ever considers its neighbors: any of the 80 other cubes where any of their coordinates differ by at most 1. For example, given the cube at `x=1,y=2,z=3,w=4`, its neighbors include the cube at `x=2,y=2,z=3,w=3`, the cube at `x=0,y=2,z=3,w=4`, and so on.

The initial state of the pocket dimension still consists of a small flat region of cubes. Furthermore, the same rules for cycle updating still apply: during each cycle, consider the number of active neighbors of each cube.

For example, consider the same initial state as in the example above. Even though the pocket dimension is 4-dimensional, this initial state represents a small 2-dimensional slice of it. (In particular, this initial state defines a 3x3x1x1 region of the 4-dimensional space.)

Simulating a few cycles from this initial state produces the following configurations, where the result of each cycle is shown layer-by-layer at each given z and w coordinate:

Before any cycles:
```
z=0, w=0
.#.
..#
###
```

After 1 cycle:
```
z=-1, w=-1
#..
..#
.#.

z=0, w=-1
#..
..#
.#.

z=1, w=-1
#..
..#
.#.

z=-1, w=0
#..
..#
.#.

z=0, w=0
#.#
.##
.#.

z=1, w=0
#..
..#
.#.

z=-1, w=1
#..
..#
.#.

z=0, w=1
#..
..#
.#.

z=1, w=1
#..
..#
.#.
```

After 2 cycles:
```
z=-2, w=-2
.....
.....
..#..
.....
.....

z=-1, w=-2
.....
.....
.....
.....
.....

z=0, w=-2
###..
##.##
#...#
.#..#
.###.

z=1, w=-2
.....
.....
.....
.....
.....

z=2, w=-2
.....
.....
..#..
.....
.....

z=-2, w=-1
.....
.....
.....
.....
.....

z=-1, w=-1
.....
.....
.....
.....
.....

z=0, w=-1
.....
.....
.....
.....
.....

z=1, w=-1
.....
.....
.....
.....
.....

z=2, w=-1
.....
.....
.....
.....
.....

z=-2, w=0
###..
##.##
#...#
.#..#
.###.

z=-1, w=0
.....
.....
.....
.....
.....

z=0, w=0
.....
.....
.....
.....
.....

z=1, w=0
.....
.....
.....
.....
.....

z=2, w=0
###..
##.##
#...#
.#..#
.###.

z=-2, w=1
.....
.....
.....
.....
.....

z=-1, w=1
.....
.....
.....
.....
.....

z=0, w=1
.....
.....
.....
.....
.....

z=1, w=1
.....
.....
.....
.....
.....

z=2, w=1
.....
.....
.....
.....
.....

z=-2, w=2
.....
.....
..#..
.....
.....

z=-1, w=2
.....
.....
.....
.....
.....

z=0, w=2
###..
##.##
#...#
.#..#
.###.

z=1, w=2
.....
.....
.....
.....
.....

z=2, w=2
.....
.....
..#..
.....
.....
```

After the full six-cycle boot process completes, 848 cubes are left in the active state.

Starting with your given initial configuration, simulate six cycles in a 4-dimensional space. How many cubes are left in the active state after the sixth cycle?

In [2]:
from itertools import product


class HigherDimensionEnergySource:
    def __init__(self, init_state, num_dimensions=4):
        self.num_dimensions = num_dimensions
        self.active_cubes =  self._get_initial_active_cubes(init_state)
    
    def _get_initial_active_cubes(self, init_state):
        active_cubes = set()
        for row_num, row in enumerate(init_state):
            for col_num, cube in enumerate(row):
                if cube == '#':
                    coords = [row_num, col_num] + [0 for i in range(self.num_dimensions - 2)]
                    active_cubes.add(tuple(coords))
        return active_cubes
    
    def next_cycle(self):
        num_active_neighbors = defaultdict(int)
        for active in self.active_cubes:
            for neighb in self._get_neighbors(active):
                num_active_neighbors[neighb] += 1
        new_active_cubes = set()
        for cube, num_act_neighbs in num_active_neighbors.items():
            if num_act_neighbs == 3:
                new_active_cubes.add(cube)
            elif num_act_neighbs == 2 and cube in self.active_cubes:
                new_active_cubes.add(cube)
        self.active_cubes = new_active_cubes
    
    def _get_neighbors(self, cube):
        neighbors = []
        diffs = (-1, 0, 1)
        self_included = product(*[[cube[pos] + d for d in diffs] for pos in range(self.num_dimensions)])
        return [c for c in self_included if c != cube]
            

def test():
    test_input = [
    '.#.',
    '..#',
    '###'
    ]
    # These 2 should behave the same way.
    e1 = EnergySource(test_input)
    e2 = HigherDimensionEnergySource(test_input, 3)
    for i in range(6):
        e1.next_cycle()
        e2.next_cycle()
    assert len(e1.active_cubes) == len(e2.active_cubes)

test()
    

def result2():
    init_state = get_input()
    e = HigherDimensionEnergySource(init_state)
    for i in range(6):
        e.next_cycle()
    return len(e.active_cubes)


result2()

1908